# Brainstorm

In [49]:
import pandas as pd
from collections import defaultdict, Counter
import json
import numpy as np

funding = pd.read_csv('clean_data.csv')
funding

,region,coordinates,fundedorg_name,fundedorg_description,fundedorg_short_description,fundedorg_categories,fundedorg_category_groups,fundedorg_permalink,announced_on,funding_type,series,money_raised_usd,n_investors,investor_name,investor_type,investor_investor_type,investor_description,investor_short_description,uuid
0,"Amsterdam, Noord-Holland, The Netherlands","(52.36999, 4.8919)",By Rocket Box,By Rocket Box is a startup that creates Welcom...,Coming Soon,[],[],rocket-box-group,2018-10-28,private_equity,NaN,NaN,1,NaN,NaN,NaN,NaN,NaN,007229b41a197ce1f42bc0e481a85861
1,"Ghent, Oost-Vlaanderen, Belgium","(51.05, 3.71667)",Wetime,Wetime is a social platform that engages coupl...,Wetime is the bridge builder for couples and f...,"['Social Network', 'Dating', 'B2C', 'Communiti...","['Internet Services', 'Community and Lifestyle...",wetime,2018-10-20,seed,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,ebe752d1cc32497faf208f52e82a208d
2,"Nairobi, Nairobi Area, Kenya","(-1.28333, 36.81667)",ImpalaCoin,We are Building a trade finance Crypto bank o...,Digital Currency Kenya,[],[],impalacoin,2018-10-20,initial_coin_offering,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,c9994563a56e4b9fb3204f1a0b5db7c3
3,"Boston, Massachusetts, United States","(42.35843, -71.05977)",MTonomy,Future of digital media: \n\n- Scalable decent...,"Blockchain infrastructure to license, distribu...","['Blockchain', 'Media and Entertainment', 'Int...","['Media and Entertainment', 'Internet Services']",mtonomy,2018-10-20,undisclosed,NaN,NaN,1,NaN,NaN,NaN,NaN,NaN,7d89c23af1a04235870605141a0d751e
4,"Philadelphia, Pennsylvania, United States","(39.95233, -75.16379)",Simply Good Jars,NaN,Eat well | Feel Good | Waste Less,['Organic Food'],['Food and Beverage'],simply-good-jars,2018-10-20,pre_seed,NaN,350000.0,2,Investor's Circle,Organization,angel_group,"Investors' Circle is the oldest, largest and m...","Investors' Circle is the oldest, largest and m...",19e3767d62d34c6487c72901dfcb2780
5,"Denver, Colorado, United States","(39.73915, -104.9847)",Sustain,NaN,Sustain rewards and incentivize employees to c...,[],[],sustain-2,2018-10-18,pre_seed,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,1db16dc217ea481d9d6101f32fa49660
6,"Tokyo, Tokyo, Japan","(35.6895, 139.69171)",Meltin MMI,Meltin MMI is a cyborg-technology company that...,Meltin MMI is a cyborg-technology company that...,"['Robotics', 'Health Care', 'Biotechnology']","['Hardware', 'Science and Engineering', 'Softw...",meltin-mmi,2018-10-17,venture,B,17974941.0,3,Dainippon Sumitomo Pharma,Organization,NaN,"""Green Prism"", the symbol of Dainippon Sumitom...",Dainippon Sumitomo Pharma is a pharmaceutical ...,f0877e36d40c4d628c35c6769cec12e3
7,"Tel Aviv, Tel Aviv, Israel","(32.08088, 34.78057)",Cognata,The fast lane to Autonomous Driving.\n\nAutono...,Cognata provides driving validation platform f...,"['Autonomous Vehicles', 'Automotive', 'Software']","['Transportation', 'Transportation', 'Software']",cognata,2018-10-17,venture,B,18500000.0,5,Scale Venture Partners,Organization,venture_capital,Scale Venture Partners invests in software com...,Scale Venture Partners is a VC firm funding th...,ee565296d58944eaaf23c7342100041b
8,"Jakarta, Jakarta Raya, Indonesia","(-6.21462, 106.84513)",Crowde,Crowde is a crowd-investing platform that enab...,Crowde is a crowd-investing platform that enab...,"['AgTech', 'Social Entrepreneurship', 'Crowdfu...","['Agriculture and Farming', 'Community and Lif...",crowde,2018-10-17,seed,NaN,NaN,2,GREE Ventures,Organization,venture_capital,GREE Ventures focuses on investing in early st...,GREE Ventures invests in early stage (Seed to ...,e8c430e2b8d7453c9f2708d27b4efcab
9,"New York, New York, United States","(40.71427, -74.00597)",WhiteSource,WhiteSource becomes part of your software deve...,WhiteSource empowers businesses to develop bet...,"['Open Source', 'Developer Tools', 'Enterprise...","['Software', 'Software', 'Software']",white-source,2018-10-17,venture,C,35000000.0,3,Susquehanna Growth Equity,Organization,private_equity_firm,"Susqueh

In [5]:
CATEGORY_CONVERSIONS = {
    '"Women\'s"': 'Women\'s',
    '"Men\'s"': 'Men\'s',
}

def get_category_counts(df):
    rows = df['fundedorg_categories']
    counts = defaultdict(int)
    for i in rows:
        clean = i.lstrip('[').rstrip(']').split(', ')
        for i in clean:
            if i == '':
                continue
            if i in CATEGORY_CONVERSIONS:
                i = CATEGORY_CONVERSIONS[i]
            counts[i.strip('\'')] += 1
    
    # Eliminate all categories that don't have at least 50 companies
    # in them from our dataset.
    return dict((k, v) for k, v in counts.iteritems() if v >= 50)

def get_categories(df):
    counts = get_category_counts(df)
    return counts.keys()

In [6]:
categories = get_categories(funding)

In [7]:
counts = get_category_counts(funding)

In [11]:
cities = defaultdict(int)
for i in funding['region']:
    cities[i] += 1

cities = dict((k, v) for k, v in cities.iteritems() if v >= 30)
Counter(cities).most_common()

[('San Francisco, California, United States', 1099),
 ('New York, New York, United States', 682),
 ('Beijing, Beijing, China', 608),
 ('Shanghai, Shanghai, China', 530),
 ('London, England, United Kingdom', 523),
 ('Boston, Massachusetts, United States', 317),
 ('Shenzhen, Guangdong, China', 311),
 ('Los Angeles, California, United States', 300),
 ('Washington, District of Columbia, United States', 205),
 ('Tel Aviv, Tel Aviv, Israel', 154),
 ('Chicago, Illinois, United States', 149),
 ('Singapore, Central Region, Singapore', 144),
 ('Seattle, Washington, United States', 141),
 ('Toronto, Ontario, Canada', 140),
 ('Bangalore, Karnataka, India', 138),
 ('Denver, Colorado, United States', 130),
 ('Austin, Texas, United States', 123),
 ('New Delhi, Delhi, India', 113),
 ('Stockholm, Stockholms Lan, Sweden', 109),
 ('San Diego, California, United States', 106),
 ('Mumbai, Maharashtra, India', 103),
 ('Berlin, Berlin, Germany', 87),
 ('Vancouver, British Columbia, Canada', 86),
 ('Sydney, N

In [43]:
city_lists = defaultdict(list)
for c, _ in counts.iteritems():
    rows = funding.loc[funding['fundedorg_categories'].str.contains(c, regex=False)]['region']
    for i in rows:
        i = str(i)
        if i not in cities:
            continue
        city_lists[i].append(c)

city_percents = defaultdict(lambda: defaultdict(float))
diff_categories = set()
for c, _ in city_lists.iteritems():
    for k, v in Counter(city_lists[c]).most_common():
        city_percents[c][k] = float(v) / counts[k]

In [50]:
json_list = []
for k, v in city_percents.iteritems():
    v = Counter(city_percents[k]).most_common()[0]
    entry = {"city": k, "category": v[0], "percent": v[1]}
    json_list.append(entry)

with open('data.json', 'w') as outfile:
    json.dump(json_list, outfile)